In [1]:
from glob import iglob
import os

import pandas as pd
import screed
import seaborn as sns
import tqdm

In [2]:
cd ~/data_sm/kmer-hashing/quest-for-orthologs/analysis/2019/

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/analysis/2019


In [3]:
ls -lha

total 6.3M
drwxr-xr-x 4 olga czb 4.0K Jan  8 17:34 ./
drwxr-xr-x 3 olga czb 4.0K Dec 25 19:12 ../
-rw-r--r-- 1 olga czb 6.2M Sep  6  2018 GO.83ebff415cfec35b9ae1888e.tsv.gz
drwxr-xr-x 2 olga czb 128K Jan  8 07:42 intermediates/
drwxr-xr-x 3 olga czb 4.0K Jan  8 17:38 transcription-factors/


# Read transcription factors 

In [4]:
tf_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_human_transcription_factors')[0]
print(tf_df.shape)
tf_df.head()

(1639, 5)


,Gene,ID,DBD,Motif status (Feb 2018)(Link to human TFs annotation),IUPAC consensus (from selected PWM)
0,AC008770.3,ENSG00000267179,C2H2 ZF,Likely sequence specific TF according to liter...,NaN
1,AC023509.3,ENSG00000267281,bZIP,Known motif – from protein with 100% identical...,RTGACGTCAY
2,AC092835.1,ENSG00000233757,C2H2 ZF,Likely sequence specific TF according to liter...,NaN
3,AC138696.1,ENSG00000264668,C2H2 ZF,Known motif – from protein with 100% identical...,RYGGAGAGTTAGC
4,ADNP,ENSG00000101126,Homeodomain,Likely sequence specific TF according to liter...,NaN


# Go to Quest for Orthologs fastas

In [5]:
cd /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


In [6]:
ls -lha

total 2.6G
drwxr-xr-x 5 olga root 4.0K Jan 10 07:42 ./
drwxr-xr-x 3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x 5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x 5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 8 olga czb   32K Jan  8 08:13 Eukaryota/
-rw-r--r-- 1 olga czb   64M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.csv
-rw-r--r-- 1 olga czb  4.0M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara_merged_uniprot.parquet
-rw-r--r-- 1 olga czb   12M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.parquet
-rw-r--r-- 1 olga czb   661 Jan  8 17:33 qfo_human_vs_opisthokont_tfs.sh
-rw-r--r-- 1 olga czb  2.6G Dec 25 18:46 QfO_release_2019_04.tar.gz
-rw-r--r-- 1 olga czb   18K May 10  2019 README
-rw-r--r-- 1 olga czb   12K Jan  8 07:47 species_metadata.csv


## Read species metadata

In [7]:
species_metadata = pd.read_csv("species_metadata.csv")
print(species_metadata.shape)
species_metadata.head()

(78, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
1,UP000000798,224324,AQUAE,1553,0,1557,Aquifex aeolicus (strain VF5),4290.0,Aquifex aeolicus,strain VF5
2,UP000006548,3702,ARATH,27475,14123,41920,Arabidopsis thaliana (Mouse-ear cress),1496.0,Arabidopsis thaliana,Mouse-ear cress
3,UP000001570,224308,BACSU,4260,7,4268,Bacillus subtilis (strain 168),4290.0,Bacillus subtilis,strain 168
4,UP000001414,226186,BACTN,4782,0,4823,Bacteroides thetaiotaomicron (strain ATCC 2914...,4290.0,Bacteroides thetaiotaomicron,strain ATCC 29148 / DSM 2079 / NCTC 10582 / E5...


### Subset to opisthokonts

In [8]:
# Estimated opisthokonta divergence time from http://timetree.org/
t = 1105
opisthokonts = species_metadata.query('divergence_from_human_mya <= @t')
print(opisthokonts.shape)
opisthokonts.head()

(35, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
5,UP000007241,684364,BATDJ,8610,0,8685,Batrachochytrium dendrobatidis (strain JAM81 /...,1105.0,Batrachochytrium dendrobatidis,strain JAM81 / FGSC 10211
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.0,Bos taurus,Bovine
8,UP000001554,7739,BRAFL,28542,2,28608,Branchiostoma floridae (Florida lancelet) (Amp...,684.0,Branchiostoma floridae,Florida lancelet
9,UP000001940,6239,CAEEL,19986,8309,28507,Caenorhabditis elegans,797.0,Caenorhabditis elegans,NaN


In [9]:
opisthokonts.query('scientific_name == "Homo sapiens"')

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
32,UP000005640,9606,HUMAN,21146,74769,96332,Homo sapiens (Human),0.0,Homo sapiens,Human


## Subset human proteins to TFs

In [10]:
human_proteome_id = 'UP000005640'

In [11]:
ll Eukaryota/human-protein-fastas/

total 1
lrwxrwxrwx 1 olga 104 Dec 26 19:44 UP000005640_9606_additional.fasta -> /mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_additional.fasta
lrwxrwxrwx 1 olga  93 Dec 26 19:44 UP000005640_9606.fasta -> /mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606.fasta


In [12]:
!head Eukaryota/human-protein-fastas/*

==> Eukaryota/human-protein-fastas/UP000005640_9606.fasta <==
>tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isoform CRA_a OS=Homo sapiens OX=9606 GN=hCG_2014768 PE=4 SV=1
MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAKVVGKGPLATGGIKK
SGKK
>sp|A0A024RBG1|NUD4B_HUMAN Diphosphoinositol polyphosphate phosphohydrolase NUDT4B OS=Homo sapiens OX=9606 GN=NUDT4B PE=3 SV=1
MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPG
GAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREW
FKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSV
R
>tr|A0A075B6H5|A0A075B6H5_HUMAN T cell receptor beta variable 20/OR9-2 (non-functional) (Fragment) OS=Homo sapiens OX=9606 GN=TRBV20OR9-2 PE=4 SV=1
METVVTTLPREGGVGPSRKMLLLLLLLGPGSGLSAVVSQHPSRVICKSGTSVNIECRSLD

==> Eukaryota/human-protein-fastas/UP000005640_9606_additional.fasta <==
>tr|A0A024QYW1|A0A024QYW1_HUMAN Isoform of A6NGB0, Transmembrane protein 191C OS=Homo sapiens OX=9606 GN=TMEM191C PE=4 SV=1
MVNNTDFLMLNNLWNKLCLVSMDFCFPLDFVSNVFWIFASKFIIVT

In [13]:
ll Eukaryota/UP000005640*

-rw-r--r-- 1 olga   36918952 Apr 12  2019 Eukaryota/UP000005640_9606_additional.fasta
-rw-r--r-- 1 olga   34929850 Apr 13  2019 Eukaryota/UP000005640_9606_DNA.fasta
-rw-r--r-- 1 olga      20163 Apr 13  2019 Eukaryota/UP000005640_9606_DNA.miss
-rw-r--r-- 1 olga   13737946 Apr 12  2019 Eukaryota/UP000005640_9606.fasta
-rw-r--r-- 1 olga      50862 May  6  2019 Eukaryota/UP000005640_9606_gcoord.miss
-rw-r--r-- 1 olga 1010405266 May  6  2019 Eukaryota/UP000005640_9606_gcoord.xml
-rw-r--r-- 1 olga    2805864 Apr 12  2019 Eukaryota/UP000005640_9606.gene2acc
-rw-r--r-- 1 olga   69939964 May  3  2019 Eukaryota/UP000005640_9606.idmapping
-rw-r--r-- 1 olga  122397065 May 10  2019 Eukaryota/UP000005640_9606.xml


In [14]:
!grep 'ENSG00000267179' Eukaryota/UP000005640*

Eukaryota/UP000005640_9606.gene2acc:ENSG00000267179	F5H0A9	ENSG00000267179
Eukaryota/UP000005640_9606.gene2acc:ENSG00000267179	K7EJH5	ENSG00000267179
Eukaryota/UP000005640_9606.gene2acc:ENSG00000267179	K7EPL6	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EJH5	BioMuta	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EJH5	Ensembl	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EJH5	GeneCards	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EPL6	BioMuta	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EPL6	Ensembl	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:K7EPL6	GeneCards	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:F5H0A9	BioMuta	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:F5H0A9	Ensembl	ENSG00000267179
Eukaryota/UP000005640_9606.idmapping:F5H0A9	GeneCards	ENSG00000267179
Eukaryota/UP000005640_9606_gcoord.xml:    <gnCoordinate ensembl_gene_id="ENSG00000267179" ensembl_transcript_id="ENST00000591944" ensembl_translation_id="ENS

## Read Gene Accession file

```
Gene mapping files (*.gene2acc)
===============================

Column 1 is a unique gene symbol that is chosen with the following order of
preference from the annotation found in:
1) Model Organism Database (MOD)
2) Ensembl or Ensembl Genomes database
3) UniProt Ordered Locus Name (OLN)
4) UniProt Open Reading Frame (ORF)
5) UniProt Gene Name
A dash symbol ('-') is used when the gene encoding a protein is unknown.

Column 2 is the UniProtKB accession or isoform identifier for the given gene
symbol. This column may have redundancy when two or more genes have identical
translations.

Column 3 is the gene symbol of the canonical accession used to represent the
respective gene group and the first row of the sequence is the canonical one.
```

In [15]:

def read_gene2acc(gene2acc, names=['maybe_ensembl_id', 'uniprot_id', 'canonical_accession']):
    df = pd.read_csv(gene2acc, sep='\t', header=None, na_values='-', names=names)
    return df

gene2acc = read_gene2acc('Eukaryota/UP000005640_9606.gene2acc')
# gene2acc = pd.read_csv('Eukaryota/UP000005640_9606.gene2acc', sep='\t', header=None, na_values='-', names=columns)
print(gene2acc.shape)
gene2acc.head()

(96332, 3)


,maybe_ensembl_id,uniprot_id,canonical_accession
0,NaN,A0A075B7B6,NaN
1,NaN,A0A075B714,NaN
2,NaN,A0A075B713,NaN
3,NaN,A0A075B712,NaN
4,NaN,A0A075B711,NaN


In [16]:
gene2acc.dropna()

,maybe_ensembl_id,uniprot_id,canonical_accession
600,ACOT7L,Q6ZUV0,ACOT7L
601,BARGIN,Q6ZT62,BARGIN
602,BARGIN,Q6ZT62-2,BARGIN
603,BCE1,O60756,BCE1
604,C1orf140,Q5VVS0,C1orf140
...,...,...,...
96327,UNQ6190/PRO20217,Q6UXQ8,UNQ6190/PRO20217
96328,UNQ6493/PRO21345,Q6UXR8,UNQ6493/PRO21345
96329,UNQ6494/PRO21346,Q6UXR6,UNQ6494/PRO21346
96330,UNQ9165/PRO28630,Q6UXU0,UNQ9165/PRO28630


## Read ID mapping file

```
Database mapping files (*.idmapping)
====================================

These files contain mappings from UniProtKB to other databases for each
reference proteome.
The format consists of three tab-separated columns:

1. UniProtKB accession
2. ID_type:
   Database name as shown in UniProtKB cross-references and supported by the ID
   mapping tool on the UniProt web site (http://www.uniprot.org/mapping)
3. ID:
   Identifier in the cross-referenced database.

```

In [17]:
id_mapping = pd.read_csv('Eukaryota/UP000005640_9606.idmapping', sep='\t', header=None, names=['uniprot_id', 'id_type', 'db_id'])
print(id_mapping.shape)
id_mapping.head()

(2668934, 3)


,uniprot_id,id_type,db_id
0,P62258-2,UniParc,UPI00001E6021
1,P62258-2,Ensembl_TRS,ENST00000571732
2,P62258-2,Ensembl_PRO,ENSP00000461762
3,P62258-2,Ensembl_TRS,ENST00000616643
4,P62258-2,Ensembl_PRO,ENSP00000481059


In [18]:
id_mapping.id_type.value_counts()

EMBL          273867
GI            270441
EMBL-CDS      198723
eggNOG        110646
UniParc        95965
               ...  
Allergome         77
PeroxiBase        33
REBASE             5
GeneDB             2
PATRIC             1
Name: id_type, Length: 68, dtype: int64

In [19]:
id_mapping.id_type.nunique()

68

### Get ENSEMBL id mapping

In [20]:
ensembl_mapping = id_mapping.query('id_type == "Ensembl"')
print(ensembl_mapping.shape)
ensembl_mapping.head()

(78145, 3)


,uniprot_id,id_type,db_id
80,P61981,Ensembl,ENSG00000170027
178,P27348,Ensembl,ENSG00000134308
279,P30443,Ensembl,ENSG00000229215
282,P30443,Ensembl,ENSG00000224320
342,Q96QU6,Ensembl,ENSG00000110455


### Get ENSEMBL TF ID mapping

In [21]:
ensembl_mapping_tfs = ensembl_mapping.query('db_id in @tf_df.ID')
print(ensembl_mapping_tfs.shape)
ensembl_mapping_tfs.head()

(5753, 3)


,uniprot_id,id_type,db_id
11056,Q6IQ32,Ensembl,ENSG00000101544
13379,P35869,Ensembl,ENSG00000106546
13744,O43918,Ensembl,ENSG00000160224
16547,O95076,Ensembl,ENSG00000156150
24705,Q14865,Ensembl,ENSG00000150347


In [22]:
tf_df_uniprot = tf_df.merge(ensembl_mapping, left_on='ID', right_on='db_id')
print(tf_df_uniprot.shape)
tf_df_uniprot.head()

(5753, 8)


,Gene,ID,DBD,Motif status (Feb 2018)(Link to human TFs annotation),IUPAC consensus (from selected PWM),uniprot_id,id_type,db_id
0,AC008770.3,ENSG00000267179,C2H2 ZF,Likely sequence specific TF according to liter...,NaN,K7EJH5,Ensembl,ENSG00000267179
1,AC008770.3,ENSG00000267179,C2H2 ZF,Likely sequence specific TF according to liter...,NaN,K7EPL6,Ensembl,ENSG00000267179
2,AC008770.3,ENSG00000267179,C2H2 ZF,Likely sequence specific TF according to liter...,NaN,F5H0A9,Ensembl,ENSG00000267179
3,AC023509.3,ENSG00000267281,bZIP,Known motif – from protein with 100% identical...,RTGACGTCAY,K7ELQ4,Ensembl,ENSG00000267281
4,AC092835.1,ENSG00000233757,C2H2 ZF,Likely sequence specific TF according to liter...,NaN,A0A087WUV0,Ensembl,ENSG00000233757


In [23]:
ensembl_mapping_tfs.uniprot_id.nunique()

5749

In [24]:
ensembl_mapping_tfs.db_id.nunique()

1633

In [26]:
# !grep ENSG Eukaryota/human-protein-fastas/* 

In [27]:
human_outdir = 'Eukaryota/human-transcription-factor-fastas/'
! mkdir $human_outdir

mkdir: cannot create directory ‘Eukaryota/human-transcription-factor-fastas/’: File exists


# Write tfs with uniprot IDs to disk

In [28]:
pwd

'/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019'

In [29]:
tf_df_uniprot.to_csv("human_transcription_factors_with_uniprot_ids.csv", index=False)
tf_df_uniprot.to_csv("human_transcription_factors_with_uniprot_ids.csv.gz", index=False)
tf_df_uniprot.to_parquet("human_transcription_factors_with_uniprot_ids.parquet", index=False)

# Read human proteins and subset to human tfs

In [45]:
tf_records = []


for filename in iglob('Eukaryota/human-protein-fastas/*.fasta'):
    with screed.open(filename) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in ensembl_mapping_tfs.uniprot_id.values:
                tf_records.append(record)
print(len(tf_records))


5749


In [46]:
tf_records[:3]

[{'name': 'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized protein OS=Homo sapiens OX=9606 PE=1 SV=1', 'sequence': 'MEPEGRGSLFEDSDLLHAGNPKENDVTAVLLTPGSQELMIRDMAEALTQWRQLNSPQGDVPEKPRNLVLLGLPISTPDVISQLEHEEELEREVSKAASQKHWETIPESKELTPEKDISEEESAPGVLIVRFSKESSSECEDSLESQQENHEKHLIQEAVTEKSSRERSYQSDEFRRNCTQRSLLVQQQGERLHHCDSFKNNLKQNSDIIRHERICAGKKPWKCNECEKAFSYYSAFVLHQRIHTGEKPYECNECGKAFSQSIHLTLHQRIHTGEKPYECHECGKAFSHRSALIRHHIIHTGEKPYECNECGKAFNQSSYLTQHQRIHTGEKPYECNECGKAFSQSTFLTQHQVIHTGEKPYKCNECGKAFSDRSGLIQHQRTHTGERPYECNECGKAFGYCSALTQHQRTHTGEKPYKCNDCAKAFSDRSALIRHQRTHTGEKPYKCKDCGKAFSQSSSLTKHQKTHTGEKPYKCKECGKAFSQSSSLSQHQKTHAGVKTKKYVQALSEHLTFGQHKRIHTG', 'description': ''},
 {'name': 'sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8 OS=Homo sapiens OX=9606 GN=E2F8 PE=1 SV=1', 'sequence': 'MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKEGSQGEPWTPTANLKMLISAVSPEIRNRDQKRGLFDNRSGLPEAKDCIHEHLSGDEFEKSQPSRKEKSLGLLCHKFLARYPNYPNPAVNNDICLDEVAEELNVERRRIYDIVNVLESLHMVSRLAKNRYTWHGRHNLNKTLGTLKSIGEENKYAEQIMMIKKKEYEQEFDFIKSYS

## Write output

In [47]:

with open(f'{human_outdir}/human_transcription_factor_proteins.fasta', 'w') as f:
    for record in tf_records:
        f.write(">{name}\n{sequence}\n".format(**record))

# Randomly subset TFs

In [32]:
tf_df_uniprot_subsampled = tf_df_uniprot.sample(100, random_state=0)
print(tf_df_uniprot_subsampled.shape)
tf_df_uniprot_subsampled.head()

(100, 8)


,Gene,ID,DBD,Motif status (Feb 2018)(Link to human TFs annotation),IUPAC consensus (from selected PWM),uniprot_id,id_type,db_id
3229,TFCP2L1,ENSG00000115112,Grainyhead,Known motif – High-throughput in vitro [942],DCYRGHNNNDDCYRGH,Q9NZI6,Ensembl,ENSG00000115112
2493,PREB,ENSG00000138073,Unknown,Likely sequence specific TF according to liter...,NaN,H7C0W3,Ensembl,ENSG00000138073
257,BPTF,ENSG00000171634,Unknown,Known motif – In vivo/Misc source [79],KKKNTTGTKKNV,J3KSY0,Ensembl,ENSG00000171634
1789,MXI1,ENSG00000119950,bHLH,Known motif – In vivo/Misc source [553],CCACGTGG,P50539,Ensembl,ENSG00000119950
1185,HNF4A,ENSG00000101076,Nuclear receptor,Known motif – High-throughput in vitro [369],VRGGTCAAAGTCCA,F8WBS7,Ensembl,ENSG00000101076


## Write CSV of random subset to disk

In [41]:
tf_df_uniprot_subsampled.to_csv("human_transcription_factors_with_uniprot_ids_random_subset100.csv", index=False)
tf_df_uniprot_subsampled.to_csv("human_transcription_factors_with_uniprot_ids_random_subset100.csv.gz", index=False)
tf_df_uniprot_subsampled.to_parquet("human_transcription_factors_with_uniprot_ids_random_subset100.parquet", index=False)

## Read human proteins and subset to human tfs

In [37]:
tfs_uniprot_ids_subset = set(tf_df_uniprot_subsampled.uniprot_id)

tf_records_subset = []

for filename in iglob('Eukaryota/human-protein-fastas/*.fasta'):
    with screed.open(filename) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in tfs_uniprot_ids_subset:
                tf_records_subset.append(record)
print(len(tf_records_subset))


100


In [38]:
tf_records_subset[:3]

[{'name': 'sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo sapiens OX=9606 GN=FAM170A PE=2 SV=1', 'sequence': 'MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQGVGEVTSTSEYCSCVSSSRKLIHSGIQRIHRDSPQPQSPLAQVQERGETPPRSQHVSLSSYSSYKTCVSSLCVNKEERGMKIYYMQVQMNKGVAVSWETEETLESLEKQPRMEEVTLSEVVRVGTPPSDVSTRNLLSDSEPSGEEKEHEERTESDSLPGSPTVEDTPRAKTPDWLVTMENGFRCMACCRVFTTMEALQEHVQFGIREGFSCHVFHLTMAQLTGNMESESTQDEQEEENGNEKEEEEKPEAKEEEGQPTEEDLGLRRSWSQCPGCVFHSPKDRNS', 'description': ''},
 {'name': 'sp|A6NJ46|NKX63_HUMAN Homeobox protein Nkx-6.3 OS=Homo sapiens OX=9606 GN=NKX6-3 PE=2 SV=1', 'sequence': 'MESNLQGTFLLNNTPLAQFPEMKAPVCQYSVQNSFYKLSPPGLGPQLAAGTPHGITDILSRPVAAPNNSLLSGYPHVAGFGGLSSQGVYYSPQVGNFSKAGNEYPTRTRNCWADTGQDWRGGRQCSNTPDPLSDSIHKKKHTRPTFTGHQIFALEKTFEQTKYLAGPERARLAYSLGMTESQVKVWFQNRRTKWRKKSALEPSSSTPRAPGGAGAGAGGDRAPSENEDDEYNKPLDPDSDDEKIRLLLRKHRAAFSVLSLGAHSV', 'description': ''},
 {'name': 'sp|A8MWA4|Z705E_HUMAN Putative zinc finger protein 705E OS=Homo sapiens OX=9606 GN=ZNF705E PE=3 SV=2', 'sequence': 'MHSLKK

## Write output

In [44]:
human_outdir_subset = 'Eukaryota/human-transcription-factor-fastas-random-subset100/'
! mkdir $human_outdir_subset

with open(f'{human_outdir_subset}/human_transcription_factor_proteins.fasta', 'w') as f:
    for record in tf_records_subset:
        f.write(">{name}\n{sequence}\n".format(**record))

mkdir: cannot create directory ‘Eukaryota/human-transcription-factor-fastas-random-subset100/’: File exists


In [43]:
ls -lha $human_outdir_subset

total 33K
drwxr-xr-x  2 olga czb 4.0K Jan 17 07:52 ./
drwxr-xr-x 12 olga czb  32K Jan 17 08:01 ../
